## 01-05 Autograd

In [2]:
import torch

torch.manual_seed(0)
torch.__version__

'1.3.0'

通过设置`retain_graph`保存计算图

In [3]:
w = torch.tensor([1.], requires_grad=True)
x = torch.tensor([2.], requires_grad=True)

a = torch.add(w, x)
b = torch.add(w, 1)
y = torch.mul(a, b)

y.backward(retain_graph=True)
print(w.grad)
y.backward()
print(w.grad)

tensor([5.])
tensor([10.])


通过使用`grad_tensors`设置梯度权重

In [5]:
w = torch.tensor([1.], requires_grad=True)
x = torch.tensor([2.], requires_grad=True)

a = torch.add(w, x)
b = torch.add(w, 1)
y0 = torch.mul(a, b)  # y0 = (x + w) * (w + 1)  dy0/dw = 5
y1 = torch.add(a, b)  # y1 = (x + w) + (w + 1)  dy1/dw = 2

loss = torch.cat([y0, y1], dim=0)  # [y0, y1]

grad_tensors = torch.tensor([1., 2.])
loss.backward(gradient=grad_tensors)  # 按照grad_tensors中的数值对loss加权
print(w.grad)

tensor([9.])


通过设置`create_graph`创建计算图，计算二阶导数

In [6]:
x = torch.tensor([3.], requires_grad=True)
y = torch.pow(x, 2)  # y = x^2

grad_1 = torch.autograd.grad(y, x, create_graph=True)
print(grad_1)

grad_2 = torch.autograd.grad(grad_1[0], x)
print(grad_2)

(tensor([6.], grad_fn=<MulBackward0>),)
(tensor([2.]),)


**Tip1：**梯度不自动清零

In [7]:
w = torch.tensor([1.], requires_grad=True)
x = torch.tensor([2.], requires_grad=True)

for _ in range(4):
    a = torch.add(w, x)
    b = torch.add(w, 1)
    y = torch.mul(a, b)

    y.backward()
    print(w.grad)
    w.grad.zero_()

tensor([5.])
tensor([5.])
tensor([5.])
tensor([5.])


**Tip2：**依赖于叶子节点的节点，`requires_grad`默认为True

In [8]:
w = torch.tensor([1.], requires_grad=True)
x = torch.tensor([2.], requires_grad=True)

a = torch.add(w, x)
b = torch.add(w, 1)
y = torch.mul(a, b)

print(a.requires_grad, b.requires_grad, y.requires_grad)

True True True


**Tip3：**叶子节点不可执行in-place

In [18]:
a = torch.ones((1,))
print(id(a), a)
a = a + torch.ones((1,))
print(id(a), a)
a +=  torch.ones((1,))  # in-place操作
print(id(a), a)
a.add_(torch.ones((1,)))  # in-place操作
print(id(a), a)

139689894784288 tensor([1.])
139689888651616 tensor([2.])
139689888651616 tensor([3.])
139689888651616 tensor([4.])


In [15]:
# 下述代码会提示RuntimeError
# w = torch.tensor([1.], requires_grad=True)
# x = torch.tensor([2.], requires_grad=True)

# a = torch.add(w, x)
# b = torch.add(w, 1)
# y = torch.mul(a, b)

# w.add_(1)

# y.backward()